In [139]:
import requests
from key import fred_key
import pandas as pd

base_url = "https://api.stlouisfed.org/fred/"
obs_endpoint = "series/observations"

start_date = "1950-01-01"
end_date = "2023-12-31"



d_gdp = {
    'name': 'GDP',
    'series_id': "GDPC1",
    'api_key': fred_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': 'a',
    'units': 'lin'
}

d_cnd = {
    'name': 'CND',
    'series_id': "PCEND",
    'api_key': fred_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': 'a',
    'units': 'lin'
}

d_cd = {
    'name': 'CD',
    'series_id': "PCEDG",
    'api_key': fred_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': 'a',
    'units': 'lin'
}

# full-time and part-time employees
d_h = {
    'name': 'H',
    'series_id': "B4701C0A222NBEA",
    'api_key': fred_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': 'a',
    'units': 'lin'
}



# excluding farm employees
d_l = {
    'name': 'L',
    'series_id': "PAYEMS",
    'api_key': fred_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': 'a',
    'units': 'lin'
}

d_AveW = {
    'name': 'AveW',
    'series_id': "AHETPI",
    'api_key': fred_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': 'a',
    'units': 'lin'
}

request_parameters = [d_gdp, d_cd, d_cnd, d_h, d_l, d_AveW]




In [141]:
def fetch_data(parameter):
    response = requests.get(base_url + obs_endpoint, params=parameter)
    if response.status_code == 200:
        res_data = response.json()
        obs_data = pd.DataFrame(res_data['observations'])
        obs_data['date'] = pd.to_datetime(obs_data['date'])
        obs_data.set_index('date', inplace=True)
        obs_data['value'] = obs_data['value'].astype(float)
        obs_data = obs_data.drop(['realtime_start', 'realtime_end'], axis=1)
        obs_data.rename(columns={'value': parameter['name']}, inplace=True)
        return obs_data
    else:
        print('Failed to retrieve data. Status code:', response.status_code)
        


dataframe = pd.DataFrame()
# concatenate the data
for parameter in request_parameters:
    df = fetch_data(parameter)
    if dataframe.empty:
        dataframe = df
    else:
        dataframe = pd.concat([dataframe, df], axis=1)

# Display the merged dataframe
dataframe

,GDP,CD,CND,H,L,AveW
date,,,,,,
1950-01-01,2458.532,NaN,NaN,100064.0,45282.0,NaN
1951-01-01,2656.320,NaN,NaN,108525.0,47926.0,NaN
1952-01-01,2764.803,NaN,NaN,110757.0,48903.0,NaN
1953-01-01,2894.412,NaN,NaN,112184.0,50304.0,NaN
1954-01-01,2877.708,NaN,NaN,107712.0,49087.0,NaN
...,...,...,...,...,...,...
2019-01-01,20692.087,1522.7,3006.5,257352.0,150906.0,23.51
2020-01-01,20234.074,1628.9,3084.2,242613.0,142165.0,24.69
2021-01-01,21407.693,2006.4,3500.2,251215.0,146276.0,25.91


In [149]:
# adjust the dataset

# add productivity
dataframe['GDP/L'] = (dataframe['GDP']/dataframe['L'])  # annual average income
dataframe['AveH'] = dataframe['H']/dataframe['L']*1000
dataset1 = dataframe.dropna()
dataset1  # data from 1964 to 2022




,GDP,CD,CND,H,L,AveW,GDP/L,AveH
date,,,,,,,,
1964-01-01,4205.277,59.5,152.7,122889.0,58394.0,2.54,0.072016,2104.479912
1965-01-01,4478.555,66.4,163.3,127604.0,60879.0,2.63,0.073565,2096.026544
1966-01-01,4773.931,71.8,177.9,133972.0,64025.0,2.73,0.074564,2092.495119
1967-01-01,4904.864,74.0,185.0,136172.0,65935.0,2.86,0.074389,2065.246076
1968-01-01,5145.914,84.8,199.8,139143.0,68027.0,3.02,0.075645,2045.408441
1969-01-01,5306.595,90.5,214.2,143024.0,70515.0,3.22,0.075255,2028.277671
1970-01-01,5316.391,90.0,228.8,140823.0,71007.0,3.40,0.074871,1983.227006
1971-01-01,5491.446,102.4,239.7,140043.0,71331.0,3.63,0.076985,1963.283846
1972-01-01,5780.048,116.4,257.4,144127.0,73788.0,3.90,0.078333,1953.257982
